In [1]:
# Import Pandas library
import pandas as pd

# Set data precision for the decimals
#pd.options.display.precision=2
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Read the Excel file
xls = pd.ExcelFile('2. Manufacturing.xlsx')

# Extract the sheet names from the excel file into a variable
companies = xls.sheet_names

# Create an empty lists: FinRatiolist / SharePricelist
FinRatiolist = []    # Long Term
SharePricelist = []  # Short term

# Temporary lists to store the pre-merger and post-merger share prices for all the companies separately
prelist = []  # Short term
postlist = []  # Short term

# Initialize an index variable with default value as Zero
idx=0

print("*******************************")
print("Data Cleaning activity started!")
print("*******************************")
# Loop the Sheet names in the Excel file
for company in companies:
    #If the sheet is for Share Price (Even index):
    if idx%2 != 0 :
        print('Cleaning Share Price data for :',company)
        #
        #   Business logic for Share Price(Short term) goes here
        #
        tdf_shareprice = pd.read_excel(xls, sheet_name=company)
        
         # Remove the columns where all rows are null
        tdf_shareprice = tdf_shareprice.loc[:, (tdf_shareprice.isnull().sum(axis=0) < len(tdf_shareprice))]
        tdf_shareprice
        tdf_shareprice.fillna(0, inplace=True)
        
        ############COLLECT PRE-MERGER RATIOS - START #############
        df_pre = tdf_shareprice.iloc[:,0:8]
        df_pre['Mergertype'] = 0 # 0-Premerger / 1-Post merger
        
        # Rename the Unnamed columns based on the Column Index
        df_pre = df_pre.rename(columns={df_pre.columns[0]: 'Date', 
                                              df_pre.columns[1]: 'Open', 
                                              df_pre.columns[2]: 'High', 
                                              df_pre.columns[3]: 'Low', 
                                              df_pre.columns[4]: 'Close', 
                                              df_pre.columns[5]: 'Volume', 
                                              df_pre.columns[6]: 'HLSpread', 
                                              df_pre.columns[7]: 'OCSpread'})
        
        # Temporary storage of the Pre-merger share prices for debugging
        #prelist.append(df_pre)
        
        # Filter the Pre-Merger share price rows where there are no values in any of the columns
        #some header columnshave "Volume" and "0" in the Volume column
        df_pre = df_pre[df_pre['Volume']!='Volume'] 
        df_pre = df_pre[df_pre['Volume']!=0]
        
        # Reformat the date column in the Pre-Merger dataframe
        df_pre['Date']= pd.to_datetime(df_pre.Date, infer_datetime_format=True, format='%d-%m-%Y')

        ############COLLECT PRE-MERGER RATIOS - END  #############
        
        
        ############COLLECT POST-MERGER RATIOS - START  #############
        df_post= tdf_shareprice.iloc[:,8:19]
        df_post['Mergertype'] = 1 #  0-Premerger / 1-Post merger
        
        # Rename the Unnamed columns based on the Column Index
        df_post = df_post.rename(columns={df_post.columns[0]: 'Date', 
                                              df_post.columns[1]: 'Open', 
                                              df_post.columns[2]: 'High', 
                                              df_post.columns[3]: 'Low', 
                                              df_post.columns[4]: 'Close', 
                                              df_post.columns[5]: 'Volume', 
                                              df_post.columns[6]: 'HLSpread', 
                                              df_post.columns[7]: 'OCSpread'})
        
        
        # Temporary storage of the Post-merger share prices for debugging
        #postlist.append(df_post)
        
        # Filter the Post-Merger share price rows where there are no values in any of the columns
        #some header columnshave "Volume" and "0" in the Volume column
        df_post = df_post[df_post['Volume']!='Volume']
        df_post = df_post[df_post['Volume']!=0]
        
        # Reformat the date column in the Post-Merger dataframe
        df_post['Date']= pd.to_datetime(df_post.Date, infer_datetime_format=True, format='%d-%m-%Y')

        ############COLLECT POST-MERGER RATIOS - END  #############       
        
        # CONCATENATE PRE & POST MERGER details into a single dataframe for a company
        df_shareprice= pd.concat([df_pre,df_post])
        
        # Add the Company information from the sheet name
        df_shareprice['Company'] = company
        
        # Add the data frame to the List (Share price list)
        SharePricelist.append(df_shareprice)
                
        #Increment the index value to navigate to the next sheet
        idx+=1
        
        # Go to Next sheet
        continue
        
    print('Cleaning Financial Ratio data for :',company)   
    #
    #   Business logic for Financial Ratios (Long term) goes here
    #
   
    #If the sheet is for Financial Ratios (Else part / Odd sheets):
    # Read the Excel based on the sheet name and load it into a temporary dataframe
    df_finratio = pd.read_excel(xls, sheet_name=company)

    # Rename the zeroth column as it is not unique in the excel (has company name attached to column header)
    df_finratio.rename(columns={ df_finratio.columns[0]: "FinRatio" }, inplace = True)

    # Add a new column to store the company name against each Ratio
    df_finratio['Company'] = company

    # Remove the columns where all rows are null
    #df_finratio = df_finratio.loc[:, (df_finratio.isnull().sum(axis=0) < len(df_finratio))]
    df_finratio=df_finratio.dropna(axis=1,how='all')
 
    # Replace null values by zero
    df_finratio.fillna(0, inplace=True)

    # Filter the rows which are not required (some header rows where there are no ratios specified)
    df_finratio = df_finratio[df_finratio['FinRatio'] !=0] 

    # Remove Pre-Mean and Post-Mean columns as it is not available in all sheets(Column Names - Unnamed: 4 & Unnamed: 9)
    if len(df_finratio.columns)==11:# where Pre-Mean and Post-Mean columns are available
        df_finratio = df_finratio.drop([df_finratio.columns[4] , df_finratio.columns[9]] , axis='columns')
        
    # Rename the Unnamed columns based on the Column Index
    df_finratio = df_finratio.rename(columns={df_finratio.columns[1]: 'Y-3', 
                                              df_finratio.columns[2]: 'Y-2', 
                                              df_finratio.columns[3]: 'Y-1', 
                                              df_finratio.columns[4]: 'Y', 
                                              df_finratio.columns[5]: 'Y+1', 
                                              df_finratio.columns[6]: 'Y+2', 
                                              df_finratio.columns[7]: 'Y+3'})
  
    # Add New Columns : PreMean, PostMean and MergeYear and default it to Zero
    df_finratio['PreMean']= df_finratio.iloc[:,1:4].mean(axis=1)
    df_finratio['PostMean']= df_finratio.iloc[:,5:8].mean(axis=1)
    df_finratio['MergeYear']=0
 
    # Add the data frame to the List (Financial Ratio list)
    FinRatiolist.append(df_finratio)

    #Increment the index value to navigate to the next sheet    
    idx+=1
    #print(idx)

    #Go to Next sheet
    continue

    
# Concatenate all share prices into a single dataframe
df_allshareprice= pd.concat(SharePricelist)

# Export the combined Share prices into an excel
df_allshareprice.to_excel('allshareprice.xlsx')
print("Share Price data export completed!")

# Concatenate all Financial Ratios into a single dataframe
df_allfinratio= pd.concat(FinRatiolist)

# Export the combined Financial ratios into an excel
df_allfinratio.to_excel('allfinratio.xlsx')
print("Financial data export completed!")

print("*********************************")
print("Data Cleaning activity completed!")
print("*********************************")
# End of Program #


*******************************
Data Cleaning activity started!
*******************************
Cleaning Financial Ratio data for : Tata Chemicals
Cleaning Share Price data for : Tata Chemicals (2)


C:\Users\prithivirajan.n\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\prithivirajan.n\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Cleaning Financial Ratio data for : Reliance Power
Cleaning Share Price data for : Reliance Power (2)
Cleaning Financial Ratio data for : Abbott
Cleaning Share Price data for : Abbott (2)
Cleaning Financial Ratio data for : JSW Steel
Cleaning Share Price data for : JSW Steel (2)
Cleaning Financial Ratio data for : Mahindra
Cleaning Share Price data for : Mahindra (2)
Cleaning Financial Ratio data for : Fortis
Cleaning Share Price data for : Fortis (2)
Cleaning Financial Ratio data for : Adani
Cleaning Share Price data for : Adani (2)
Cleaning Financial Ratio data for : ONGC Videsh
Cleaning Share Price data for : ONGC
Cleaning Financial Ratio data for : Rain
Cleaning Share Price data for : Rain (2)
Cleaning Financial Ratio data for : Hero
Cleaning Share Price data for : Hero (2)
Cleaning Financial Ratio data for : Sun Pharma
Cleaning Share Price data for : Sun Pharma (2)
Cleaning Financial Ratio data for : Piramal Enterprises
Cleaning Share Price data for : Piramal
Cleaning Financial Ra

In [1]:
FinRatiolist[4]

NameError: name 'FinRatiolist' is not defined

In [97]:
prelist[0]

,Date,Open,High,Low,Close,Volume,HLSpread,OCSpread,Mergertype
0,Historic Data for the Period : 01-Jan-2009 to ...,0,0,0,0,0,0,0,0
1,Pre-Merger,0,0,0,0,0,0,0,0
2,Period High : 341.80,Period Low : 99.70,Change in market-cap : 96.34%,0,0,0,0,0,0
3,Date,Open,High,Low,Close,Volume,- SPREAD -,0,0
4,0,0,0,0,0,0,(High-Low),(Open-Close),0
5,31-12-2009,336,341.80,320,322.20,266778,21.80,13.80,0
6,30-12-2009,322.80,336.80,321.10,332.85,279495,15.70,-10.05,0
7,29-12-2009,323.40,324,320.30,321.70,91584,3.70,1.70,0
8,24-12-2009,324.40,324.85,320,322.05,93636,4.85,2.35,0
9,23-12-2009,315.25,322.90,315,320.50,82926,7.90,-5.25,0


In [98]:
postlist[0]

,Date,Open,High,Low,Close,Volume,HLSpread,OCSpread,Mergertype
0,Historic Data for the Period : 01-Jan-2011 to ...,0,0,0,0,0,0,0,1
1,Post Merger,0,0,0,0,0,0,0,1
2,Period High : 412.00,Period Low : 287.80,Change in market-cap : -21.18%,0,0,0,0,0,1
3,Date,Open,High,Low,Close,Volume,- SPREAD -,0,1
4,0,0,0,0,0,0,(High-Low),(Open-Close),1
5,30-12-2011,311,314,305.05,311.35,17570,8.95,-0.35,1
6,29-12-2011,308.90,313.20,303.95,308.65,19504,9.25,0.25,1
7,28-12-2011,312,312.60,302.85,306.85,30029,9.75,5.15,1
8,27-12-2011,316,318.25,311.25,312.65,13825,7,3.35,1
9,26-12-2011,317.90,318.45,315.10,317,13272,3.35,0.90,1
